<a href="https://colab.research.google.com/github/adhish-bahl/2347203_NNDL_Lab/blob/main/2347203_ETE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Importing Libraries**

#### **Mounting Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### **Importing Dataset**